# Prototyping

Creates a neural network which evaluates a time series and produce a set of predicted values for the time series
Predicted values may be used in a policy to make a trade. This policy may be modeled by simple multiple regression or a neural network.

## Data
Test data is taken as most recent to avoid lookahead bias. Train data is split into a validation and training set during fitting.


## TODO
- Convert feature percentages to stdev
- Adding VIX as a signal
- Adding High/Low as signals
- Multiple securities/ aggregate samples
- Policy network
- Regularization (l2)
- Dilated convolution

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import pandas_datareader as pdr
from datetime import datetime

def from_network(symbol):
    return pdr.get_data_yahoo(symbols=symbol, start=datetime(1900, 1, 1))

def from_file(symbol):
    dataset_path = keras.utils.get_file("{}.csv".format(symbol), "http://localhost:8000/data/daily/{}.csv".format(symbol))
    column_names = ['Date','Open','High','Low','Close','Adj Close','Volume'] 
    return pd.read_csv(dataset_path, 
                              names=column_names, 
                              dtype={'Close': np.float64,'Open': np.float64,'High': np.float64,'Adj Close': np.float64, 'Volume': np.float64},
                              header=0,
                              na_values = "?", 
                              comment='\t',
                              sep=",",
                              skipinitialspace=True)

#dataset = raw_dataset.copy()
dataset = from_network('SPY')
dataset = dataset.sort_values(by=['Date'],ascending=False)




In [2]:
print dataset

                  High         Low        Open       Close       Volume  \
Date                                                                      
2019-01-22  265.059998  261.059998  264.820007  262.859985  115355500.0   
2019-01-18  266.980011  263.000000  264.980011  266.459991  127900300.0   
2019-01-17  263.920013  259.959991  260.010010  262.959991   96118400.0   
2019-01-16  261.970001  260.600006  260.829987  260.980011   77636700.0   
2019-01-15  260.700012  257.809998  257.820007  260.350006   85208300.0   
2019-01-14  258.299988  256.410004  256.859985  257.399994   70908200.0   
2019-01-11  259.010010  257.029999  257.679993  258.980011   73858100.0   
2019-01-10  259.160004  255.500000  256.260010  258.880005   96823900.0   
2019-01-09  258.910004  256.190002  257.559998  257.970001   95006600.0   
2019-01-08  257.309998  254.000000  256.820007  256.769989  102512600.0   
2019-01-07  255.949997  251.690002  252.690002  254.380005  103139100.0   
2019-01-04  253.110001  2

In [22]:

dataset_stats = dataset.describe()
dataset_stats = dataset_stats.transpose()
NUM_INPUT_NEURONS = 64
NUM_OUTPUT_NEURONS = 1
NUM_SAMPLES = len(dataset)
NUM_TEST_SAMPLES = int(.25 * NUM_SAMPLES)
NUM_TEST_SAMPLES = 1000
DAY_OFFSET = 5
print NUM_SAMPLES
# Create features (only close price for now)
def convert_to_percentage(old, new):
    return (old - new) / old

def convert_labels_to_category(labels): 
    # Simplification - If positive return, 1, else 0
    # return map(lambda arr: 1 if arr[0] > 1 else 0, labels)
    # rounding simpliciation
    return map(lambda arr: map(lambda val: round(val,4),arr), labels)

def convert_to_train(raw_dataset):
    dataset = raw_dataset.copy()
    features = []
    labels = []
    for i in range(DAY_OFFSET, len(dataset) - NUM_INPUT_NEURONS):

        feature_dataset = dataset[i:i+NUM_INPUT_NEURONS].copy()
        latest_close = feature_dataset['Close'].iloc[0]
        
        features.append(
            feature_dataset['Close']
                .map(lambda current: convert_to_percentage(latest_close, current))
                .tolist()
        )
        labels.append([
            dataset['Close'].iloc[i-1] / latest_close, # 1 day trade
#             dataset['Close'].iloc[i-2] / latest_close, # 2 day trade
#             dataset['Close'].iloc[i-3] / latest_close, # 3 day trade
#             dataset['Close'].iloc[i-4] / latest_close, # 4 day trade
#             dataset['Close'].iloc[i-5] / latest_close, # 5 day trade
        ])
        
    # Without converting labels the precision is hard to determine accuracy. 
    # Rather than crude 0/1, maybe this can be more sophisticated
    labels = convert_labels_to_category(labels)
    
    return [features,labels]
converted_feature_set = convert_to_train(dataset)
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(converted_feature_set[0][0])]),
    layers.Dense(32, kernel_regularizer=keras.regularizers.l1_l2(l1=0.0001, l2=0.0001), activation=tf.nn.relu),
    layers.Dense(16, kernel_regularizer=keras.regularizers.l1_l2(l1=0.0001, l2=0.0001), activation=tf.nn.relu),
    #layers.Dense(16, activation=tf.nn.relu),
    layers.Dense(NUM_OUTPUT_NEURONS)
  ])

  model.compile(loss='mse',
                optimizer='sgd',
                metrics=[
                    'mae',
                ])
  return model

tensorboard = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

model = build_model()
model.summary()

6542
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 6,785
Trainable params: 6,785
Non-trainable params: 0
_________________________________________________________________


In [23]:
#len(converted_feature_set[0][0])
print NUM_TEST_SAMPLES
train_data = np.array(converted_feature_set[0][NUM_TEST_SAMPLES:])
train_labels = np.array(converted_feature_set[1][NUM_TEST_SAMPLES:])

test_data = np.array(converted_feature_set[0][:NUM_TEST_SAMPLES])
test_labels = np.array(converted_feature_set[1][:NUM_TEST_SAMPLES])


1000


In [24]:
print train_data
print train_labels

[[ 0.         -0.00551303  0.00922086 ...  0.04166468  0.04898282
   0.0599112 ]
 [ 0.          0.0146531   0.01960218 ...  0.05419706  0.06506552
   0.05836972]
 [ 0.          0.00502267  0.00714003 ...  0.0511621   0.04436672
   0.06293086]
 ...
 [ 0.          0.00280309  0.00630583 ... -0.00840926 -0.00911114
  -0.00490651]
 [ 0.          0.00351259  0.00983838 ... -0.01194772 -0.00773127
   0.00281097]
 [ 0.          0.00634809  0.00775852 ... -0.0112835  -0.0007041
   0.00141044]]
[[1.0023]
 [0.9945]
 [1.0149]
 ...
 [0.9965]
 [1.0028]
 [1.0035]]


In [ ]:

history = model.fit(
    train_data, train_labels,
    epochs=90, validation_split = 0.2, verbose=1,
    callbacks=[tensorboard]
)

Train on 4378 samples, validate on 1095 samples
Epoch 1/90
4378/4378 [==============================] - 0s 90us/step - loss: 0.1249 - mean_absolute_error: 0.1588 - val_loss: 0.0410 - val_mean_absolute_error: 0.0151
Epoch 2/90
4378/4378 [==============================] - 0s 41us/step - loss: 0.0413 - mean_absolute_error: 0.0180 - val_loss: 0.0408 - val_mean_absolute_error: 0.0115
Epoch 3/90
4378/4378 [==============================] - 0s 44us/step - loss: 0.0411 - mean_absolute_error: 0.0155 - val_loss: 0.0407 - val_mean_absolute_error: 0.0101
Epoch 4/90
4378/4378 [==============================] - 0s 43us/step - loss: 0.0410 - mean_absolute_error: 0.0145 - val_loss: 0.0406 - val_mean_absolute_error: 0.0095
Epoch 5/90
4378/4378 [==============================] - 0s 41us/step - loss: 0.0409 - mean_absolute_error: 0.0141 - val_loss: 0.0406 - val_mean_absolute_error: 0.0092
Epoch 6/90
4378/4378 [==============================] - 0s 41us/step - loss: 0.0408 - mean_absolute_error: 0.0139 - v

In [ ]:
outputs = model.predict(test_data)

In [ ]:
print test_data[0]
print test_labels[0]
print outputs[0]

In [ ]:
# primitive policy temporarily in place of a RL policy network

# signal step for our policy network
signals = map(lambda arr: sum(arr) / len(arr),outputs)
# signals = map(lambda arr: arr[0],outputs)
trades = map(lambda signal: 1 if round(signal,4) > 1 else 0,signals)

In [ ]:
outputs[1:]

In [ ]:
# one day return
actual = map(lambda arr: arr[0],test_labels)

df = pd.DataFrame({
    'signal':signals,
    'actual': actual,
    'trade':trades,
})

In [ ]:
df['entry_success'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 1) else 0,axis=1)
df['entry_failure'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 1) else 0,axis=1)
df['avoid_success'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 0) else 0,axis=1)
df['avoid_failure'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 0) else 0,axis=1)


In [ ]:
# primitive policy - replace with a policy network which maximizes reward
def label_success (row):
    return 0 if (row['entry_failure'] == 1 or row['entry_failure'] == 1) else 1

success = df.apply (lambda row: label_success (row),axis=1)
df['success'] = success;
df

In [ ]:
print '\nPrecision'
print sum(df['entry_success']) * 1.00 / (sum(df['entry_success']) + sum(df['entry_failure'])) 

print '\nRecall'
print sum(df['entry_success']) * 1.00 / (sum(df['entry_success']) + sum(df['avoid_failure']))

print '\nAccuracy'
print sum(df['entry_success']) * 1.00 / (NUM_TEST_SAMPLES)

print '\nNon-loss events'
print sum(df['success'])
print sum(df['success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nLose trades'
print sum(df['entry_failure'])
print sum(df['entry_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nWin trades'
print sum(df['entry_success'])
print sum(df['entry_success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nMissed opportunities'
print sum(df['avoid_failure'])
print sum(df['avoid_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nBullets dodged'
print sum(df['avoid_success'])
print sum(df['avoid_success']) / (NUM_TEST_SAMPLES * 1.00)


In [ ]:
sum(df['entry_success'])

In [ ]:
df1 = dataset.copy().iloc[DAY_OFFSET-1:].head(NUM_TEST_SAMPLES)
df2 = df.copy()

In [ ]:
pd.concat([df1.reset_index(),df2], axis=1).drop(['High','Low','Open','Volume','Adj Close'], axis=1)